In [16]:
import os
import json
import pandas as pd
import traceback
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv 
load_dotenv()

True

In [17]:
import google.generativeai as genai
API_KEY = os.getenv("GEMNI_API_KEY")
genai.configure(api_key=API_KEY)


In [18]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",google_api_key=API_KEY,)
response = llm.invoke("Tell me a short story.")
print(response.content)

The lighthouse keeper, Silas, had only two loves: the sea and his records. Every night, as the beam cut through the inky blackness, he'd spin a different record, filling the lonely tower with melodies. One night, a storm raged, the wind howling like a banshee. Silas, battling the tempest to keep the light burning, heard a different kind of sound over the storm's fury - a faint, ethereal singing.

He dismissed it as the wind playing tricks, but the singing persisted, weaving its way through the crashing waves and the mournful strains of his record, a melancholic waltz. Curiosity piqued, Silas ventured out onto the wind-swept balcony. The singing was clearer now, coming from the churning waves below.

He strained his eyes and saw it - a figure clinging to a piece of wreckage, a woman with long, seaweed-strewn hair, her voice a siren's call in the storm. He knew the risks, the treacherous currents, but he couldn't leave her to drown.

Securing a rope, Silas descended into the maelstrom. T

In [11]:
import google.generativeai as genai
import os
for model in genai.list_models():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thi

In [19]:
llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001265BAA5850>, default_metadata=())

In [21]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
import PyPDF2

In [108]:
response_json = {
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer here",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer here",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer here",
    }
}

In [109]:
TEMPLATE = """
Test:{text}
You are an expert MCQ Maker. Given the above text, it is your job to 
create a quiz of {numbers} multiple choice question for {subject} students in {tone} tone.
Make sure the question are not repeated and check all the questions to be configured the text as well.
Make sure to format your response like response_json below and use it as a guide.
Ensure to make {numbers} mcqs
### response_json
{response_json}

"""

In [110]:
quiz_genration_prompt = PromptTemplate(
    input_variables=["text","numbers","subject","tone","response_json"],
    template=TEMPLATE
)

In [111]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_genration_prompt,output_key="quiz",verbose=True)

In [112]:
TEMPLATE2 = """
You are an expert english grammerian and writer . Given a multiple choice quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 word for complexity.
If the quiz is not as per with thw congnitive and analytical abilities of the student,
update the quiz question which need to be changed and cjange the tone such that it perfectly fit the student.
QUIZ_MCQ:
{quiz}

check for expert english writer of the above quiz
"""

In [113]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE2)

In [114]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [115]:
genrate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain],
                      input_variables=["text","numbers","subject","tone","response_json"],
                      output_variables=["subject","quiz"],
                      verbose = True

                      
                      )

In [116]:
file_path = "D:\GenAI\Projects\mcqGenrator\data.txt"
with open(file_path,"r") as f:
    TEXT = f.read()

In [117]:
# serilize the python dict into json formatted string
json.dumps(response_json)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer here"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer here"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer here"}}'

In [118]:
NUMBER = 5
SUBJECT = "machine learning",
TONE="simple"

In [119]:
resp = genrate_evaluate_chain(
   { "text":TEXT,
    "numbers":NUMBER,
    "subject":SUBJECT,
    "tone":TONE,
    "response_json": json.dumps(response_json)
    }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Test:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used b

In [121]:
resp

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [122]:
quiz = resp.get('quiz').strip("```json").strip("```").strip()

In [126]:
quiz_dict = json.loads(quiz)

In [128]:
quiz_table_data = []
for key , value in quiz_dict.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option}: {option_value}" for option,option_value in value["options"].items()
        ]
    )

    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq,"choices":options,"correct":correct})
    

In [129]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'choices': 'a: Donald Hebb | b: Walter Pitts | c: Arthur Samuel | d: Alan Turing',
  'correct': 'c'},
 {'MCQ': "Donald Hebb's book, 'The Organization of Behavior,' introduced a theoretical neural structure based on:",
  'choices': 'a: Mathematical models of neural networks | b: Interactions among nerve cells | c: Punched tape memory | d: Pattern classification',
  'correct': 'b'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'choices': 'a: The Hebb Machine | b: Cybertron | c: The Turing Device | d: The Samuel Analyzer',
  'correct': 'b'},
 {'MCQ': "According to Tom M. Mitchell's definition, what are the three key elements involved in machine learning?",
  'choices': 'a: Algorithms, data, and accuracy | b: Experience, tasks, and performance | c: Neurons, networks, and learning | d: Classification, prediction, and modeling',
  'correct': 'b'},
 {'MCQ': 'What a

In [130]:
import pandas as pd

In [132]:
quiz_data = pd.DataFrame(quiz_table_data)

In [133]:
quiz_data.to_csv("machinglearning.csv",index=False)